### Imports

In [ ]:
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import CSVLogger, Callback
from datetime import datetime
import keras.backend as K
import extras.ourUtils as utils
import numpy as np
import Models
import sys
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import KFold
from extras.ourUtils import reverse_data_split, create_data_split

### Init

In [7]:
batch_size = 64
nrEpochs = 10
full_train = True
path = '/home/jaskmo/Documents/programering/02456DomainAdaptation/'
source_data = path + 'newTaperImg/Physionet/'
target_data = path + 'newTaperImg/Hospital/'
stdout_cell = sys.stdout
MIQ = ['DA', 'target', 'source']
kf = KFold(n_splits = 10)
n_subjects_phys = np.arange(1,21);
n_subjects_hosp = np.arange(1,35);

class FlipControle(Callback):
    def __init__(self, alphaIn):
        self.alpha = alphaIn
        print(K.get_value(lamFunk))
        
    def on_epoch_end(self, epoch, logs={}):
        p = (epoch+1)/nrEpochs
        K.set_value(self.alpha, (2/(1+np.exp(-10*p)))-1)
        print(K.get_value(lamFunk))


## Get data as generators

In [3]:
datagen = ImageDataGenerator(rescale=1./255)

# make a data generator for dplInput
def train_gen_DAnet(source, target, batch_size):
    half = batch_size//2
    while True:
        source_data, source_lable = source.next()
        target_data, target_lable = target.next()
        if len(source_lable) != batch_size or len(target_lable) != batch_size:
            continue
        dpl_data = np.concatenate((source_data[:half,...],target_data[:half,...]),axis=0)
               
        domain_tmp = np.ones(batch_size, dtype='int8')
        domain_tmp[half:] = domain_tmp[half:] * 0
        dpl_lable = np.concatenate((domain_tmp.reshape(batch_size,1),
                                       np.flip(domain_tmp,0).reshape(batch_size,1)),1)

        yield({'lplInput':source_data,'dplInput':dpl_data}, {'lplOut':source_lable,'dplOut':dpl_lable})
        
def test_gen_DAnet(source, target, batch_size):
    half = batch_size//2
    while True:
        source_data, source_lable = source.next()
        target_data, target_lable = target.next()
        if len(source_lable) != batch_size or len(target_lable) != batch_size:
            continue
        dpl_data = np.concatenate((source_data[:half,...],target_data[:half,...]),axis=0)
               
        domain_tmp = np.ones(batch_size, dtype='int8')
        domain_tmp[half:] = domain_tmp[half:] * 0
        dpl_lable = np.concatenate((domain_tmp.reshape(batch_size,1),
                                       np.flip(domain_tmp,0).reshape(batch_size,1)),1)

        yield({'lplInput':target_data,'dplInput':dpl_data}, {'lplOut':target_lable,'dplOut':dpl_lable})

In [4]:
whitelist = dir()
whitelist.append('whitelist')
whitelist.append('phys_split')
whitelist.append('hosp_split')
whitelist.append('this')

In [5]:
for phys_split, hosp_split in zip(kf.split(n_subjects_phys),kf.split(n_subjects_hosp)):

    # memory management
    this = sys.modules[__name__]
    for n in dir():
        if n not in whitelist: delattr(this, n)
    K.clear_session()

    #Define current time
    now = datetime.now()
    #print("PHYS:\n" + "Train:" + str(phys_split[0]) + '\n' + 'Test:' + str(phys_split[1]) + "\n")
    #print("Hosp:\n" + "Train:" + str(hosp_split[0]) + '\n' + 'Test:' + str(hosp_split[1]) + "\n")
    
    reverse_data_split(source_data)
    reverse_data_split(target_data)
    
    #Data split for physionet
    create_data_split(source_data,phys_split[1]+1)
    #Data split for target
    create_data_split(target_data,hosp_split[1]+1)
    
    
    #### Train data


    train_gen_source = datagen.flow_from_directory(source_data + '/train', target_size=(224, 224), 
                                                   batch_size=batch_size, class_mode='categorical', shuffle=True)

    train_gen_target = datagen.flow_from_directory(target_data + '/train', target_size=(224, 224), 
                                                   batch_size=batch_size, class_mode='categorical', shuffle=True)

    train_gen_DA = train_gen_DAnet(train_gen_source, train_gen_target, batch_size)

    train_stepE = np.floor_divide(train_gen_source.n, batch_size)
    train_stepE_target = np.floor_divide(train_gen_target.n, batch_size)
    #### validation data

    valid_gen_source = datagen.flow_from_directory(source_data + '/validation', target_size=(224, 224), 
                                                   batch_size=batch_size, class_mode='categorical', shuffle=True)
    valid_gen_target = datagen.flow_from_directory(target_data + '/validation', target_size=(224, 224), 
                                                   batch_size=batch_size, class_mode='categorical', shuffle=True)

    valid_gen_DA = test_gen_DAnet(valid_gen_source, valid_gen_target, batch_size)

    valid_stepE = np.floor_divide(valid_gen_source.n, batch_size)
    valid_stepE_target = np.floor_divide(valid_gen_target.n, batch_size)

    #### test data

    test_gen_source = datagen.flow_from_directory(source_data + '/test', target_size=(224, 224), 
                                                   batch_size=batch_size, class_mode='categorical', shuffle=True)
    test_gen_target = datagen.flow_from_directory(target_data + '/test', target_size=(224, 224), 
                                                   batch_size=batch_size, class_mode='categorical', shuffle=True)

    test_gen_DA = test_gen_DAnet(test_gen_source, test_gen_target, batch_size)

    test_stepE = np.floor_divide(test_gen_source.n, batch_size)
    test_stepE_target = np.floor_divide(test_gen_target.n, batch_size)   
    
    
    #Define lamFunk
    lamFunk = K.variable(0.0)
    
    for item in MIQ:
        if item=="DA":         
            current_model = Models.DA_model(lamFunk,do_rate_dpl=0, do_rate_lpl=0.5, vgg_train=False, nrUnits=[512,512])
        else:
            current_model = Models.lable_model(do_rate=0.5, vgg_train=False, nrUnits=[512,512])
        
        csv_logger = CSVLogger('/media/jaskmo/ELEK/bme/Project02456/trainingLog/' + item + 
                        str(now.day) + '-' + str(now.month) + '-' + str(now.year) + '_' + 
                        str(now.hour) + str(now.minute) + '.log')
        if item=='DA':
            current_model.fit_generator(train_gen_DA, train_stepE, epochs=nrEpochs, verbose=1, validation_data=valid_gen_DA, 
                            validation_steps=valid_stepE, callbacks=[csv_logger,FlipControle(lamFunk)], initial_epoch=0,
                            max_queue_size=5, class_weight = {'lplOut':[1,0.33,0.33,0.33,0.5],'dplOut':[1,1]})
        elif item=='source':
            current_model.fit_generator(train_gen_source, train_stepE, epochs=nrEpochs, verbose=1, validation_data=valid_gen_source, 
                            validation_steps=valid_stepE, callbacks=[csv_logger], initial_epoch=0,
                            max_queue_size=5, class_weight = [1,0.33,0.33,0.33,0.5])
        else:
            current_model.fit_generator(train_gen_target, train_stepE_target, epochs=nrEpochs, verbose=1, validation_data=valid_gen_target, 
                            validation_steps=valid_stepE_target, callbacks=[csv_logger], initial_epoch=0,
                            max_queue_size=5, class_weight = [1,0.33,0.33,0.33,0.5])
                     
        if item == "DA":
            DAlpm = utils.dissect_DAlpm(current_model)
            current_model = DAlpm
            
#         #Save the model
#         current_model.save(filepath=path + 'models/'+ item + str(now.day) + '-' + str(now.month) + '-' + str(now.year) + '_' + 
#                        str(now.hour) + str(now.minute) + '.h5')
        
        #TEST
        DTD_dic = {'Physionet':test_gen_source,'Hospital':test_gen_target}
        # save to file 
        test_file = '/media/jaskmo/ELEK/bme/Project02456/testLog/'+ item + str(now.day) + '-' + str(now.month) + '-' + str(now.year) + '_' + str(now.hour) + str(now.minute) + '.log'
        sys.stdout = open(test_file, 'w')
        
        for dom in ['Physionet', 'Hospital']:
            test_img, test_lable = DTD_dic[dom].next()
            for count in range(DTD_dic[dom].n//batch_size):
                tmp_img, tmp_lable = DTD_dic[dom].next()
                test_img = np.concatenate((test_img, tmp_img), axis=0)
                test_lable = np.concatenate((test_lable, tmp_lable),axis=0)

            # Compute the test metrecis 
            inv_map = {v: k for k, v in DTD_dic[dom].class_indices.items()}
            target_names = list(inv_map.values())

            targets_test_int = [np.where(r == 1)[0][0] for r in test_lable]
            y_pred = current_model.predict(test_img)
            y_pred2 = np.argmax(y_pred, axis = 1)
            # Test accuracy:
            acc = accuracy_score(targets_test_int, y_pred2)
            
            conf_mat = confusion_matrix(targets_test_int, y_pred2)
            # Per class metrics
            class_report = classification_report(targets_test_int, y_pred2, target_names=target_names)

            print('Accuracy on ' + dom + ' data = ' + str(acc) +'\n \n' + 
                  'Confution matric on ' + dom + ' data: \n' + str(conf_mat) + '\n\n' + 
                  'Class report on ' + dom + ' data: \n' + class_report + '\n\n\n\n')

        sys.stdout = stdout_cell

Found 23124 images belonging to 5 classes.
Found 17038 images belonging to 5 classes.
Found 2414 images belonging to 5 classes.
Found 3118 images belonging to 5 classes.
Found 2816 images belonging to 5 classes.
Found 2793 images belonging to 5 classes.
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lplInput (InputLayer)            (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
dplInput (InputLayer)            (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
model_1 (Model)                  (None, 25088)         14714688    lplInput[0][0]                   
                                       

Epoch 1/10
372/372 [==============================] - 266s - loss: 20.9079 - lplOut_loss: 1.0860 - dplOut_loss: 0.2820 - lplOut_categorical_accuracy: 0.5717 - dplOut_categorical_accuracy: 0.8968 - val_loss: 18.8157 - val_lplOut_loss: 2.2584 - val_dplOut_loss: 0.1210 - val_lplOut_categorical_accuracy: 0.3625 - val_dplOut_categorical_accuracy: 0.9521
Epoch 2/10
372/372 [==============================] - 265s - loss: 15.2402 - lplOut_loss: 0.6581 - dplOut_loss: 0.1336 - lplOut_categorical_accuracy: 0.7586 - dplOut_categorical_accuracy: 0.9537 - val_loss: 15.4804 - val_lplOut_loss: 2.5770 - val_dplOut_loss: 0.1142 - val_lplOut_categorical_accuracy: 0.3719 - val_dplOut_categorical_accuracy: 0.9536
Epoch 3/10
372/372 [==============================] - 265s - loss: 12.3302 - lplOut_loss: 0.5871 - dplOut_loss: 0.1355 - lplOut_categorical_accuracy: 0.7835 - dplOut_categorical_accuracy: 0.9556 - val_loss: 13.2774 - val_lplOut_loss: 2.5516 - val_dplOut_loss: 0.1394 - val_lplOut_categorical_accura

/usr/local/lib/python3.5/dist-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
vgg16 (Model)                multiple                  14714688  
_________________________________________________________________
preLp (Flatten)              (None, 25088)             0         
_________________________________________________________________
preDnsDo (Dropout)           (None, 25088)             0         
_________________________________________________________________
lpl1 (Dense)                 (None, 512)               12845568  
_________________________________________________________________
lpl1Do (Dropout)             (None, 512)               0         
_________________________________________________________________
lpl2 (Dense)                 (None, 512)               262656    
__________

351/351 [==============================] - 128s - loss: 11.2131 - categorical_accuracy: 0.5662 - val_loss: 9.3947 - val_categorical_accuracy: 0.7773
Epoch 2/10
351/351 [==============================] - 126s - loss: 8.6647 - categorical_accuracy: 0.7517 - val_loss: 7.8399 - val_categorical_accuracy: 0.7855
Epoch 3/10
351/351 [==============================] - 126s - loss: 7.3824 - categorical_accuracy: 0.7772 - val_loss: 6.8199 - val_categorical_accuracy: 0.8138
Epoch 4/10
351/351 [==============================] - 126s - loss: 6.5403 - categorical_accuracy: 0.7946 - val_loss: 6.1354 - val_categorical_accuracy: 0.8230
Epoch 5/10
351/351 [==============================] - 126s - loss: 5.9320 - categorical_accuracy: 0.8058 - val_loss: 5.6141 - val_categorical_accuracy: 0.8257
Epoch 6/10
351/351 [==============================] - 126s - loss: 5.4548 - categorical_accuracy: 0.8122 - val_loss: 5.2037 - val_categorical_accuracy: 0.8245
Epoch 7/10
351/351 [==============================] - 12

...

365/365 [==============================] - 262s - loss: 12.2968 - lplOut_loss: 0.5785 - dplOut_loss: 0.1259 - lplOut_categorical_accuracy: 0.7836 - dplOut_categorical_accuracy: 0.9570 - val_loss: 14.2531 - val_lplOut_loss: 3.5720 - val_dplOut_loss: 0.1255 - val_lplOut_categorical_accuracy: 0.2685 - val_dplOut_categorical_accuracy: 0.9670
Epoch 4/10
365/365 [==============================] - 261s - loss: 10.4976 - lplOut_loss: 0.5432 - dplOut_loss: 0.1827 - lplOut_categorical_accuracy: 0.7966 - dplOut_categorical_accuracy: 0.9342 - val_loss: 12.8667 - val_lplOut_loss: 3.5664 - val_dplOut_loss: 0.2238 - val_lplOut_categorical_accuracy: 0.2876 - val_dplOut_categorical_accuracy: 0.9197
Epoch 5/10
365/365 [==============================] - 261s - loss: 9.3012 - lplOut_loss: 0.5256 - dplOut_loss: 0.2482 - lplOut_categorical_accuracy: 0.8016 - dplOut_categorical_accuracy: 0.9039 - val_loss: 11.6110 - val_lplOut_loss: 3.3803 - val_dplOut_loss: 0.2058 - val_lplOut_categorical_accuracy: 0.2947 -

365/365 [==============================] - 133s - loss: 11.1302 - categorical_accuracy: 0.5763 - val_loss: 9.5241 - val_categorical_accuracy: 0.7506
Epoch 2/10
365/365 [==============================] - 131s - loss: 8.5670 - categorical_accuracy: 0.7578 - val_loss: 7.9185 - val_categorical_accuracy: 0.7707
Epoch 3/10
365/365 [==============================] - 131s - loss: 7.2767 - categorical_accuracy: 0.7836 - val_loss: 6.9445 - val_categorical_accuracy: 0.7810
Epoch 4/10
365/365 [==============================] - 131s - loss: 6.4290 - categorical_accuracy: 0.7988 - val_loss: 6.2265 - val_categorical_accuracy: 0.7860
Epoch 5/10
365/365 [==============================] - 131s - loss: 5.8191 - categorical_accuracy: 0.8084 - val_loss: 5.7097 - val_categorical_accuracy: 0.7925
Epoch 6/10
365/365 [==============================] - 131s - loss: 5.3269 - categorical_accuracy: 0.8158 - val_loss: 5.2510 - val_categorical_accuracy: 0.7932
Epoch 7/10
365/365 [==============================] - 13

349/349 [==============================] - 126s - loss: 4.3660 - categorical_accuracy: 0.8265 - val_loss: 4.3316 - val_categorical_accuracy: 0.8021
Epoch 10/10
349/349 [==============================] - 126s - loss: 4.0682 - categorical_accuracy: 0.8312 - val_loss: 4.0338 - val_categorical_accuracy: 0.8106
Found 21643 images belonging to 5 classes.
Found 18527 images belonging to 5 classes.
Found 3637 images belonging to 5 classes.
Found 1975 images belonging to 5 classes.
Found 3042 images belonging to 5 classes.
Found 2342 images belonging to 5 classes.
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lplInput (InputLayer)            (None, 224, 224, 3)   0                                            
____________________________________________________________________________________________________
dplInput (InputLayer)            (

Epoch 1/10
353/353 [==============================] - 250s - loss: 21.0115 - lplOut_loss: 1.0612 - dplOut_loss: 0.2727 - lplOut_categorical_accuracy: 0.5768 - dplOut_categorical_accuracy: 0.9067 - val_loss: 19.0503 - val_lplOut_loss: 2.2728 - val_dplOut_loss: 0.1271 - val_lplOut_categorical_accuracy: 0.3982 - val_dplOut_categorical_accuracy: 0.9702
Epoch 2/10
353/353 [==============================] - 249s - loss: 15.4485 - lplOut_loss: 0.6399 - dplOut_loss: 0.1344 - lplOut_categorical_accuracy: 0.7615 - dplOut_categorical_accuracy: 0.9516 - val_loss: 15.6730 - val_lplOut_loss: 2.5133 - val_dplOut_loss: 0.1423 - val_lplOut_categorical_accuracy: 0.4167 - val_dplOut_categorical_accuracy: 0.9555
Epoch 3/10
353/353 [==============================] - 248s - loss: 12.5287 - lplOut_loss: 0.5713 - dplOut_loss: 0.1280 - lplOut_categorical_accuracy: 0.7880 - dplOut_categorical_accuracy: 0.9554 - val_loss: 13.5467 - val_lplOut_loss: 2.6075 - val_dplOut_loss: 0.1418 - val_lplOut_categorical_accura

ZeroDivisionError: integer division or modulo by zero